## 1. Introduction

As per statista.com, economic loss due to natural disaster events worldwide amounted to about 1.5 trillion US dollars in the last five years. It
becomes important for any individual or business to safeguard themselves from huge financial loss caused by an unforeseen event like fire, terrorism or any natural disasters like Earthquake, Flood, Windstorm, Tornado, Hail, Hurricane, or loss due to machinery breakdown in case of production plants. Providing protection against such financial loss by charging a nominal premium compared to property value is called Property Insurance. It is estimated that the global commercial insurance market is projected to reach from USD 692.33 Billion in 2020 to USD 1,227.8 Billion by 2028.

From property insurer perspective, it is important to price the premiums effectively against each peril and for various occupancy types depending on their expected financial loss to be competitive in the market and be profitable at the same time. So, they employ Catastrophe Risk Modeling analyst to scrape the client’s property details, run the predictive model, asses the risk to identify the vulnerability and severity of property damage to generate the expected loss and charge accordingly for various catastrophe perils

In real world, Catastrophe risk modeling combines historical disaster information with current demographics, building, and financial data to determine the potential financial impact of catastrophe in various geographic locations. However, the process of developing sophisticated catastrophe models is complex and draws on expertise from a broad range of technical and financial disciplines. Hence, we will limit ourselves to one peril Earthquake and just use one type of occupancy ‘residential’ in this case study for educational purpose and realize the power of machine learning in predictive analytics.

We will preprocess and explore an historical earthquake event data and build a machine learning model to identify the severity of damage to buildings caused by an earthquake. Additionally, we will also merge household conditions data to capture the socio-economic and demographics of affected areas (learned from https://www.sciencedirect.com/science/article/pii/S2212094718301385) and see if they improve score in our classification task. This is useful for Property Insurers to assign different rates based on the expected severity of damage to client property that is identified by our machine learning model.


## 2. Problem Statement

Given the geo location, structure details of building and household socio-economic demographics information predict the severity of damage to buildings caused by Earthquake

## 3. Business Constraint

- Interpretability is important to some extent
- No strict low latency concerns
- High accuracy. Errors affect pricing of premiums and can cost writing business
- Predicting probability of point belonging to each class is not necessary

## 4. Data Overview

(Source: Nepal Earthquake 2015 open data portal (link: https://reliefweb.int/report/nepal/open-data-portal-2015-earthquake-launched-national-planning-commission))
The data mainly contains information on the structure of buildings like the construction material used for foundation/roof, height of building, plinth area, surface, age, their legal ownership, geo location and household conditions, income level, etc. to name a few for the eleven severely affected districts of Nepal. Features are self-explanatory by their names. We will explore each feature in EDA notebook.

## 5. Performance Metric

- Both precision and recall are important so Micro-F1 score is good choice as data is imbalanced and we care about overall accuracy
- Confusion matrix, recall matrix and precision matrix to see how our model is performing on train and test data for each class
- Classification report to see how our model is performing on each class


## Import the required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import shutil
import os
import time
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import re

## Import the data

In [ ]:
filepath = '/content/drive/MyDrive/Assignments/Self Case Study -1 (Nepal Earthquake)/'


In [ ]:
# Unzipping data, preparing file structure with required files
if not os.path.isdir('data'):
    os.makedirs('data')

    # Building structure and ownership data
    shutil.copy(filepath+'eq2015_buildings.zip', '/content/')
    !unzip eq2015_buildings.zip
    os.rename('building_damage_assessment_building_ownership_and_use_building_structure', 'data/buildings/')
    os.remove('data/buildings/csv_building_damage_assessment.csv')
    os.remove('eq2015_buildings.zip')

    # Household socio-economic demographics data
    shutil.copy(filepath+'eq2015_households.zip', '/content/')
    !unzip eq2015_households.zip
    os.rename('household_demographics_household_earthquake_impact_household_resources', 'data/households/')
    os.remove('data/households/csv_household_earthquake_impact.csv')
    os.remove('data/households/ward_vdcmun_district_name_mapping.csv')
    os.remove('eq2015_households.zip')

Archive:  eq2015_buildings.zip
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/csv_building_damage_assessment.csv  
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/csv_building_ownership_and_use.csv  
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/csv_building_structure.csv  
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/ward_vdcmun_district_name_mapping.csv  
Archive:  eq2015_households.zip
  inflating: household_demographics_household_earthquake_impact_household_resources/csv_household_demographics.csv  
  inflating: household_demographics_household_earthquake_impact_household_resources/csv_household_earthquake_impact.csv  
  inflating: household_demographics_household_earthquake_impact_household_resources/csv_household_resources.csv  
  inflating: household_demographics_household_earthquake_impact_household_resources/ward_vdcmun_d

In [ ]:
# Preparing file structure with required files

# Building structure and ownership data
!unzip data/eq2015_buildings.zip
os.rename('building_damage_assessment_building_ownership_and_use_building_structure', 'data/buildings/')
os.remove('data/buildings/csv_building_damage_assessment.csv')

# Household socio-economic demographics data
!unzip data/eq2015_households.zip
os.rename('household_demographics_household_earthquake_impact_household_resources', 'data/households/')
os.remove('data/households/csv_household_earthquake_impact.csv')
os.remove('data/households/ward_vdcmun_district_name_mapping.csv')

Archive:  data/eq2015_buildings.zip
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/csv_building_damage_assessment.csv  
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/csv_building_ownership_and_use.csv  
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/csv_building_structure.csv  
  inflating: building_damage_assessment_building_ownership_and_use_building_structure/ward_vdcmun_district_name_mapping.csv  
Archive:  data/eq2015_households.zip
  inflating: household_demographics_household_earthquake_impact_household_resources/csv_household_demographics.csv  
  inflating: household_demographics_household_earthquake_impact_household_resources/csv_household_earthquake_impact.csv  
  inflating: household_demographics_household_earthquake_impact_household_resources/csv_household_resources.csv  
  inflating: household_demographics_household_earthquake_impact_household_resources/war

### 1. Building structure and ownership data

In [ ]:
# Buildig structure data
df_structure = pd.read_csv('data/buildings/csv_building_structure.csv')

# Building ownership data
df_ownership = pd.read_csv('data/buildings/csv_building_ownership_and_use.csv')

# Merging the both
df_buildings = pd.merge(df_structure, df_ownership, on=['building_id', 'district_id', 'vdcmun_id', 'ward_id'])
df_buildings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762106 entries, 0 to 762105
Data columns (total 44 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   building_id                             762106 non-null  int64  
 1   district_id                             762106 non-null  int64  
 2   vdcmun_id                               762106 non-null  int64  
 3   ward_id                                 762106 non-null  int64  
 4   count_floors_pre_eq                     762106 non-null  int64  
 5   count_floors_post_eq                    762106 non-null  int64  
 6   age_building                            762106 non-null  int64  
 7   plinth_area_sq_ft                       762106 non-null  int64  
 8   height_ft_pre_eq                        762106 non-null  int64  
 9   height_ft_post_eq                       762106 non-null  int64  
 10  land_surface_condition                  7621

Observation:
  1. Features like count_floors_post_eq, height_ft_post_eq, condition_post_eq are data collected post the earthquake event
  2. Target variables 'damage_grade' and 'technical_solution_proposed' have few missing values which cannot be used and count families feature has only 2 missing values.

    We will drop them

In [ ]:
# Dropping NAN value data points and post event details except target variables
df_buildings = df_buildings.drop(['count_floors_post_eq', 'height_ft_post_eq', 'condition_post_eq'], axis=1)
df_buildings = df_buildings.dropna().reset_index().drop(['index'], axis=1)

#### Identifying Target variable

In [ ]:
target_data_pro = df_buildings[['damage_grade', 'technical_solution_proposed']].value_counts(normalize=True)
target_data_pro

damage_grade  technical_solution_proposed
Grade 5       Reconstruction                 0.361609
Grade 4       Reconstruction                 0.218406
Grade 3       Major repair                   0.132913
Grade 2       Minor repair                   0.097089
Grade 1       No need                        0.067295
              Minor repair                   0.035357
Grade 3       Reconstruction                 0.033589
Grade 4       Major repair                   0.022586
Grade 2       Major repair                   0.013624
Grade 3       Minor repair                   0.012421
Grade 2       Reconstruction                 0.003143
              No need                        0.000640
Grade 1       Major repair                   0.000504
              Reconstruction                 0.000262
Grade 4       Minor repair                   0.000227
Grade 5       Major repair                   0.000188
Grade 3       No need                        0.000073
Grade 5       Minor repair              

In [ ]:
print(target_data_pro[:6])
print('\nPercentage of data points considered: ', round(target_data_pro[:6].sum()*100, 2), '%')

damage_grade  technical_solution_proposed
Grade 5       Reconstruction                 0.361609
Grade 4       Reconstruction                 0.218406
Grade 3       Major repair                   0.132913
Grade 2       Minor repair                   0.097089
Grade 1       No need                        0.067295
              Minor repair                   0.035357
dtype: float64

Percentage of data points considered:  91.27 %


As per 91.27% of data,
we can see that Reconstruction is suggested for damage grades of Grade 4 and Grade 5, Major repair for Grade 3 and Minor repair or No repair for Grade 2 and Grade 1 respectively.
Hence, we will categorize our target variable 'severity_of_damage' into 3 classes 'Mild', 'Moderate' and 'Severe'

In [ ]:
# Target Mapping
target_mapping = {'Grade 1': 'Mild', 'Grade 2': 'Mild', 'Grade 3': 'Moderate', 'Grade 4': 'Severe', 'Grade 5': 'Severe'}

df_buildings['severity_of_damage'] = df_buildings['damage_grade'].map(target_mapping)
df_buildings = df_buildings.drop(['damage_grade', 'technical_solution_proposed'], axis=1)

#### Handling float datatype

Lets see if float datatype data is continuous or discrete

In [ ]:
features_float_dtypes = df_buildings.select_dtypes('float')

for feature in features_float_dtypes:
    print(feature, '\n', df_buildings[feature].unique())

count_families 
 [ 1.  0.  2.  3.  4.  5.  6.  9.  7.  8. 11.]
has_secondary_use 
 [0. 1.]


As they are integers (discrete values). Lets map these features to integer

In [ ]:
for feature in features_float_dtypes:
    df_buildings[feature] = df_buildings[feature].astype(int)

In [ ]:
df_buildings.head(5)

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,severity_of_damage
0,120101000011,12,1207,120703,1,9,288,9,Flat,Other,...,0,0,0,0,0,0,0,0,0,Moderate
1,120101000021,12,1207,120703,1,15,364,9,Flat,Other,...,0,0,0,0,0,0,0,0,0,Severe
2,120101000031,12,1207,120703,1,20,384,9,Flat,Other,...,0,0,0,0,0,0,0,0,0,Mild
3,120101000041,12,1207,120703,1,20,312,9,Flat,Other,...,0,0,0,0,0,0,0,0,0,Mild
4,120101000051,12,1207,120703,1,30,308,9,Flat,Other,...,0,0,0,0,0,0,0,0,0,Mild


In [ ]:
df_buildings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762093 entries, 0 to 762092
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             762093 non-null  int64 
 1   district_id                             762093 non-null  int64 
 2   vdcmun_id                               762093 non-null  int64 
 3   ward_id                                 762093 non-null  int64 
 4   count_floors_pre_eq                     762093 non-null  int64 
 5   age_building                            762093 non-null  int64 
 6   plinth_area_sq_ft                       762093 non-null  int64 
 7   height_ft_pre_eq                        762093 non-null  int64 
 8   land_surface_condition                  762093 non-null  object
 9   foundation_type                         762093 non-null  object
 10  roof_type                               762093 non-null 

### 2. Socio-economic demographics data


In [ ]:
# Household demographics data
df_demographics = pd.read_csv('data/households/csv_household_demographics.csv')

# Household resources data
df_resources = pd.read_csv('data/households/csv_household_resources.csv')

df_households = pd.merge(df_demographics, df_resources, on= ['household_id', 'district_id', 'vdcmun_id', 'ward_id'])
df_households.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747365 entries, 0 to 747364
Data columns (total 43 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   household_id                                  747365 non-null  int64  
 1   district_id                                   747365 non-null  int64  
 2   vdcmun_id                                     747365 non-null  int64  
 3   ward_id                                       747365 non-null  int64  
 4   gender_household_head                         747363 non-null  object 
 5   age_household_head                            747363 non-null  float64
 6   caste_household                               747137 non-null  object 
 7   education_level_household_head                747137 non-null  object 
 8   income_level_household                        747137 non-null  object 
 9   size_household                                74

Observation:
  1. Building_id to be fetched from household_id
  2. We see that there are some features data are collected post earthquake event. We will drop them

In [ ]:
# Fetching Building_id
df_households['building_id'] = ((df_households.household_id/100).astype(int))

# Dropping Post Earthquake event
for feat in df_households.columns:
    if '_post_' in feat:
        df_households = df_households.drop([feat], axis=1)

df_households.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747365 entries, 0 to 747364
Data columns (total 28 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   household_id                                  747365 non-null  int64  
 1   district_id                                   747365 non-null  int64  
 2   vdcmun_id                                     747365 non-null  int64  
 3   ward_id                                       747365 non-null  int64  
 4   gender_household_head                         747363 non-null  object 
 5   age_household_head                            747363 non-null  float64
 6   caste_household                               747137 non-null  object 
 7   education_level_household_head                747137 non-null  object 
 8   income_level_household                        747137 non-null  object 
 9   size_household                                74

We can see that there are some entries with missing values. Let's handle them

In [ ]:
df_households[df_households['caste_household'].isna()]

,household_id,district_id,vdcmun_id,ward_id,gender_household_head,age_household_head,caste_household,education_level_household_head,income_level_household,size_household,...,has_asset_computer_pre_eq,has_asset_internet_pre_eq,has_asset_telephone_pre_eq,has_asset_mobile_phone_pre_eq,has_asset_fridge_pre_eq,has_asset_motorcycle_pre_eq,has_asset_four_wheeler_family_use_pre_eq,has_asset_four_wheeler_commercial_use_pre_eq,has_asset_none_pre_eq,building_id
88,21240200016101,21,2107,210703,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,212402000161
4035,12060200005201,12,1207,120710,Male,36.0,NaN,NaN,NaN,4.0,...,0,0,0,0,0,0,0,0,0,120602000052
17429,12270600007102,12,1205,120509,Male,33.0,NaN,NaN,NaN,5.0,...,0,0,0,0,0,0,0,0,0,122706000071
63974,20250602242103,20,2005,200506,Male,26.0,NaN,NaN,NaN,1.0,...,0,0,0,0,0,0,0,0,0,202506022421
64626,20250603272101,20,2005,200506,Male,40.0,NaN,NaN,NaN,5.0,...,0,0,0,0,0,0,0,0,0,202506032721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724011,36500209109101,36,3608,360807,Male,35.0,NaN,NaN,NaN,3.0,...,0,0,0,0,0,0,0,0,0,365002091091
724793,36500303001102,36,3608,360806,Male,31.0,NaN,NaN,NaN,3.0,...,0,0,0,0,0,0,0,0,0,365003030011
724812,36500303018102,36,3608,360806,Male,19.0,NaN,NaN,NaN,2.0,...,0,0,0,0,0,0,0,0,0,365003030181
725260,36500401102101,36,3608,360805,Male,19.0,NaN,NaN,NaN,5.0,...,0,0,0,0,0,0,0,0,0,365004011021


Most of the feature values are missing for all these 228 entries. So we will drop them instead of imputing as we have good number of data

In [ ]:
df_households = df_households.dropna()

#### Handling float datatype

In [ ]:
features_float_dtypes = df_households.select_dtypes('float')

for feature in features_float_dtypes:
    print(feature, '\n', df_households[feature].unique())

age_household_head 
 [ 31.  62.  51.  48.  70.  52.  27.  44.  34.  35.  38.  46.  49.  63.
  71.  84.  59.  55.  61.  57.  43.  65.  50.  32.  58.  54.  60.  42.
  74.  41.  53.  69.  56.  37.  73.  30.  45.  87.  82.  78.  25.  81.
  28.  29.  80.  64.  36.  90.  33.  24.  20.  40.  39.  72.  79.  67.
  75.  22.  47.  66.  76.  19.  26.  68.  23.  21.  85.  89.  77.  83.
  18.  88.  16.  17.  15.  86.  11.  14.  13.  99.  95.  91.  93.  12.
  96. 112.  97. 100.  92.  94. 110. 114.  10. 101. 107. 120. 106. 109.
  98. 103. 104. 115. 102. 119. 117. 105. 122.]
size_household 
 [ 3.  6. 13.  5.  8.  9.  4.  1.  2.  7. 10. 12. 11. 14. 21. 16. 15. 24.
 17. 18. 26. 19. 20. 22. 23. 25. 29. 32. 36. 31. 28. 37. 27. 30. 40. 34.
 35. 39.]
is_bank_account_present_in_household 
 [0. 1.]


As the above features are discrete and not continuous we will map to integer

In [ ]:
for feature in features_float_dtypes:
    df_households[feature] = df_households[feature].astype(int)

#### Handling mutli-households data

Some buildings like apartment or rental building can have more than one households. We will look at number of such buildings

In [ ]:
is_multihousehold = df_households['building_id'].groupby(by=df_households.building_id).agg('count') > 1

print('Number of buildings with more than 1 household: ', is_multihousehold.sum())
print('Number of unique buildings: ', df_households.building_id.nunique())

Number of buildings with more than 1 household:  47025
Number of unique buildings:  690457


In [ ]:
df_households.head(5)

,household_id,district_id,vdcmun_id,ward_id,gender_household_head,age_household_head,caste_household,education_level_household_head,income_level_household,size_household,...,has_asset_computer_pre_eq,has_asset_internet_pre_eq,has_asset_telephone_pre_eq,has_asset_mobile_phone_pre_eq,has_asset_fridge_pre_eq,has_asset_motorcycle_pre_eq,has_asset_four_wheeler_family_use_pre_eq,has_asset_four_wheeler_commercial_use_pre_eq,has_asset_none_pre_eq,building_id
0,12010100001101,12,1207,120703,Male,31,Rai,Illiterate,Rs. 10 thousand,3,...,0,0,0,1,0,0,0,0,0,120101000011
1,12010100002101,12,1207,120703,Female,62,Rai,Illiterate,Rs. 10 thousand,6,...,0,0,0,1,0,0,0,0,0,120101000021
2,12010100003101,12,1207,120703,Male,51,Gharti/Bhujel,Illiterate,Rs. 10 thousand,13,...,0,0,0,1,0,0,0,0,0,120101000031
3,12010100004101,12,1207,120703,Male,48,Gharti/Bhujel,Illiterate,Rs. 10 thousand,5,...,0,0,0,1,0,0,0,0,0,120101000041
4,12010100005101,12,1207,120703,Male,70,Gharti/Bhujel,Illiterate,Rs. 10 thousand,8,...,0,0,0,1,0,0,0,0,0,120101000051


## Combining building and household dataframes

In [ ]:
df_cleaned = pd.merge(df_buildings, df_households, on=['building_id', 'ward_id', 'vdcmun_id', 'district_id']).drop(['building_id', 'household_id'], axis=1)
df_cleaned.head(5)

,district_id,vdcmun_id,ward_id,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,...,has_asset_cable_pre_eq,has_asset_computer_pre_eq,has_asset_internet_pre_eq,has_asset_telephone_pre_eq,has_asset_mobile_phone_pre_eq,has_asset_fridge_pre_eq,has_asset_motorcycle_pre_eq,has_asset_four_wheeler_family_use_pre_eq,has_asset_four_wheeler_commercial_use_pre_eq,has_asset_none_pre_eq
0,12,1207,120703,1,9,288,9,Flat,Other,Bamboo/Timber-Light roof,...,0,0,0,0,1,0,0,0,0,0
1,12,1207,120703,1,15,364,9,Flat,Other,Bamboo/Timber-Light roof,...,0,0,0,0,1,0,0,0,0,0
2,12,1207,120703,1,20,384,9,Flat,Other,Bamboo/Timber-Light roof,...,0,0,0,0,1,0,0,0,0,0
3,12,1207,120703,1,20,312,9,Flat,Other,Bamboo/Timber-Light roof,...,0,0,0,0,1,0,0,0,0,0
4,12,1207,120703,1,30,308,9,Flat,Other,Bamboo/Timber-Light roof,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747124 entries, 0 to 747123
Data columns (total 62 columns):
 #   Column                                        Non-Null Count   Dtype 
---  ------                                        --------------   ----- 
 0   district_id                                   747124 non-null  int64 
 1   vdcmun_id                                     747124 non-null  int64 
 2   ward_id                                       747124 non-null  int64 
 3   count_floors_pre_eq                           747124 non-null  int64 
 4   age_building                                  747124 non-null  int64 
 5   plinth_area_sq_ft                             747124 non-null  int64 
 6   height_ft_pre_eq                              747124 non-null  int64 
 7   land_surface_condition                        747124 non-null  object
 8   foundation_type                               747124 non-null  object
 9   roof_type                                     747124 non-nu

Our final data contains 61 features excluding target variable and datasize reduced from 762106 to 747124 after cleaning

## Exporting cleaned data to drive

In [ ]:
df_cleaned.to_csv(filepath+'df_cleaned.csv', index=False)